Inteiro aleatório

In [ ]:
import secrets

def generate_num():
    # Gera um inteiro aleatório de 1024 bits
    n = secrets.randbits(1024)

    # Garante que ele tenha exatamente 1024 bits e seja ímpar:
    n |= (1 << 1023)    # define o bit mais significativo
    n |= 1              # define o bit 0 para torná-lo ímpar

    return n


def primes_up_to(n: int) -> list[int]:
    if n < 2: 
        return []
    
    is_prime = [True] * (n + 1)
    is_prime[0] = is_prime[1] = False

    # Marca múltiplos de cada primo como compostos
    lim = int(n**0.5)
    for p in range(2, lim + 1):
        if is_prime[p]:
            for multiple in range(p * p, n + 1, p):
                is_prime[multiple] = False

    # Coleta aqueles que permaneceram True
    return [i for i, prime in enumerate(is_prime) if prime]


def is_divisible_by_small_primes(n: int, primes: list[int]) -> bool:
    for p in primes:
        if p * p > n:
            break

        if n % p == 0:
            return n != p
        
    return False
    

def likely_prime_miller_rabin(n: int, k: int = 40):
    s, d = 0, n-1
    while d % 2 == 0:
        s += 1
        d //= 2

    for _ in range(k):
        a = secrets.randbelow(n-3) + 2  # base aleatória em [2, n-2]
        x = pow(a, d, n)
        if x in (1, n-1):
            continue
        for _ in range(s-1):
            x = pow(x, 2, n)
            if x == n-1:
                break
        else:
            return False
        
    return True

def generate_prime():
    primes_up_to_2000 = primes_up_to(2000)

    for _ in range(10_000):
        num = generate_num()
        if is_divisible_by_small_primes(num, primes_up_to_2000):
            continue

        if likely_prime_miller_rabin(num, 40):
            return num
        
    raise TimeoutError("Failed to generate prime after 50k iterations.")

In [33]:
num = generate_prime()
num

132969991645413115086731765141296196556073117841327563220755524396879773796629336394602678758629813152277058823390705691896160054431466674540597801714896636350325448064334899567341498138503752296158413913701894368368686943963066281660834317286923214700737732647223160305236049350625574239539860494953737927343

# RSA

In [52]:
class PublicKey:
    def __init__(self, p: int, q: int):
        self.p = p
        self.q = q

        n = p * q
        e = 65537

        self.key = (n, e)

    def verify(self, message: int, signature: int):
        return self.encrypt(signature) == message

    def encrypt(self, message: int):
        n, e = self.key
        return pow(message, e, n)


class PrivateKey:
    def __init__(self, p: int, q: int):
        self.p = p
        self.q = q

        n = p * q
        phi = (p - 1) * (q - 1)
        e = 65537
        d = pow(e, -1, phi)  # e^-1 (mod phi)

        self.key = (n, d)

    def sign(self, message: int):
        return self.decrypt(message)

    def decrypt(self, encrypted: int):
        n, d = self.key
        return pow(encrypted, d, n)

    def derive_public_key(self):
        return PublicKey(self.p, self.q)

    @staticmethod
    def generate():
        return PrivateKey(generate_prime(), generate_prime())

In [53]:
private_key = PrivateKey.generate()
public_key = private_key.derive_public_key()

encrypted = public_key.encrypt(42)
print(encrypted)
print(private_key.decrypt(encrypted))

9802800770874020263726328074971390033149990873237056087711617728222194915529507993867258910533701886328964328626983837967592099330677210671439232867589501727063617132224544587097525582064592395048314834251028834582194346275539865243522500937865312843989872827663825136844465443254576784032171925052393445719723676076086770376908106410776378517346477196044721130385370626103157834925044942115706598483090390075293371833754461787480545634412055083140544821534239414562582044549363675436645539107093717045174526681071335075256219423680379200678772303848447378619313137196513863548725756184261772836271167143908927280290
42


In [58]:
private_key = PrivateKey.generate()
public_key = private_key.derive_public_key()

signature = private_key.sign(42)
print(signature)
print(public_key.verify(42, signature))

8851725765535431813821541256208142707732301542596334628163123986410958208183189863455739180316042691342676365241388379514685595391265252859647639175620594984199699946133720975540183889559669966026918193785598684146818402345942346343386068703781939875619071527405757843442729918138632330795000171521318003675132581506788596840634257298113586531730525266053454320412931226657509802558827418170615178079895329209096695642051583562888587121729520573152611125470648081810369323537592454392928582477123004066084135261440660353332043724626182026186633297145561617806876684141324455529471892836766469045155744349746184334085
True


In [ ]:
private_key.decrypt(cypher)

42

In [45]:
signature = private_key.sign(42)

In [36]:
p = generate_prime()
q = generate_prime()
n = p * q

phi = (p - 1) * (q - 1)
e = 65537
d = pow(e, -1, phi) # e^-1 (mod phi)

public_key = (n, e)
private_key = (n, d)

In [ ]:
def generate_keys():
    p = generate_prime()
    q = generate_prime()
    n = p * q

    phi = (p - 1) * (q - 1)
    e = 65537
    d = pow(e, -1, phi) # e^-1 (mod phi)

    public_key = (n, e)
    private_key = (n, d)

In [37]:
def encrypt(public_key: tuple[int, int], message: int):
    n, e = public_key
    return pow(message, e, n)

def decrypt(private_key: tuple[int, int], cypher: int):
    n, d = private_key
    return pow(cypher, d, n)

In [40]:
decrypt(private_key, encrypt(public_key, 76))

76

# OAEP

In [ ]:
import hashlib


def sha3_256_hash(data: bytes):
    sha3_256_hasher = hashlib.sha3_256()
    sha3_256_hasher.update(data)
    
    return sha3_256_hasher.digest()

In [ ]:
def generate_datablock()

# OAEP